In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings 
import jieba
warnings.filterwarnings('ignore')

In [2]:
with open('news_classify_data.txt','r',encoding='utf-8') as f:
    txt = f.readlines()
f.close()
jieba.add_word('亦舒')
jieba.add_word('昆凌')
jieba.add_word('周杰伦')
text = np.array([_.split('_!_') for _ in txt])
text[0][0] = '6660024717287621123'
text[:,1] = [int(_) for _ in text[:,1]]
text[:,0] = [int(_) for _ in text[:,0]]
df = pd.DataFrame(text,columns = ['id','class','name','text'])
stop_word_f = open('ChineseStopWords.txt',encoding='utf-8')
lines = stop_word_f.readlines()
stop_word_f.close()
stop_list = [_[:-1] for _ in lines]
df['text'][0]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Javon\AppData\Local\Temp\jieba.cache
Loading model cost 0.386 seconds.
Prefix dict has been built successfully.


'老祖宗俗语：“万恶淫为首”，下一句更是精华，却没几个人能做到\n'

In [3]:
def dict_proc(data):
    data['text'] = pd.Series(' '.join(jieba.cut(_[:-1],HMM=True)) for _ in data['text'])
    return data
df = dict_proc(df)

In [4]:
def str2str(data,stop_list):
    for idx in range(len(data['text'])):
        char_list = []
        for char in data['text'][idx].split():
            if char not in stop_list:
                char_list.append(char)
        data['text'][idx] = ''.join(char_list)
    return data
df = str2str(df,stop_list)

In [5]:
df

,id,class,name,text
0,6660024717287621123,0,文化,老祖宗俗语万恶淫为首一句更是精华没几个人能
1,6675222202004013572,0,文化,老照片1907年山东省泰安府
2,6679940295825555980,0,文化,亦舒经典语录100句
3,6676963806968545799,0,文化,乐山大佛整修花脸清洗干净网友美完容变帅
4,6664825683656573448,0,文化,7000年前女子花样年华遭受残忍
...,...,...,...,...
56816,6691485644243337735,8,国际,外媒苏丹军队开枪驱散示威者伤军方抗议者指责
56817,6691479002646839822,8,国际,特朗普说中国经济不好外交部中国经济主管部门
56818,6691483762988941835,8,国际,美伊局势升级美国会白宫作出解释
56819,6691481494600286727,8,国际,中国女留学生澳失踪月警方呼吁公众协助寻人


In [5]:
def char2list(data):
    return [char for char in data]

df['text'].apply(char2list)

0        [老, 祖, 宗, 俗, 语, 万, 恶, 淫, 为, 首, 一, 句, 更, 是, 精, ...
1               [老, 照, 片, 1, 9, 0, 7, 年, 山, 东, 省, 泰, 安, 府]
2                           [亦, 舒, 经, 典, 语, 录, 1, 0, 0, 句]
3        [乐, 山, 大, 佛, 整, 修, 花, 脸, 清, 洗, 干, 净, 网, 友, 美, ...
4         [7, 0, 0, 0, 年, 前, 女, 子, 花, 样, 年, 华, 遭, 受, 残, 忍]
                               ...                        
56816    [外, 媒, 苏, 丹, 军, 队, 开, 枪, 驱, 散, 示, 威, 者, 伤, 军, ...
56817    [特, 朗, 普, 说, 中, 国, 经, 济, 不, 好, 外, 交, 部, 中, 国, ...
56818        [美, 伊, 局, 势, 升, 级, 美, 国, 会, 白, 宫, 作, 出, 解, 释]
56819    [中, 国, 女, 留, 学, 生, 澳, 失, 踪, 月, 警, 方, 呼, 吁, 公, ...
56820    [自, 带, 桌, 子, 来, 店, 免, 单, 英, 国, 这, 家, 餐, 厅, 目, ...
Name: text, Length: 56821, dtype: object

In [14]:
def charFreqCount(data):
    char2idx = dict()
    idx2char = dict()
    char_freq = dict()
    char_freq['<Paddling>'] = 1
    char_freq['<UNK>'] = 1
    char_freq['<EOS>'] = 1
    for chars in data:
        for i in chars:
            char_freq[i] = char_freq.get(i,0) + 1
    return char2idx,idx2char,char_freq

In [7]:
char2idx,idx2char,char_freq = charFreqCount(df['text'])

In [8]:
# char_freq = dict(sorted(char_freq.items(),key=lambda x:x[1],reverse = True))

In [17]:
def worddictProc(char2idx,idx2char,char_freq,min_count = 1,max_count = 15000):
    idx = 0
    for _,(char,count) in enumerate(char_freq.items()):
        if min_count <= count <= max_count:
            char2idx[char] = idx
            idx2char[idx] = char
            idx += 1
    return char2idx,idx2char
char2idx,idx2char = worddictProc(char2idx,idx2char,char_freq)

In [21]:
def seq2list(data,char2idx=char2idx):
    for index in range(len(data)):
        word_list = []
        word = data[index]
        for i in word:
            if i in char2idx.keys():
                word_list.append(char2idx[i])
            else:
                word_list.append(char2idx['<UNK>'])
        data[index] = word_list
    return data

In [22]:
df['text'] = seq2list(df['text'])

In [23]:
df

,id,class,name,text
0,6660024717287621123,0,文化,"[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ..."
1,6675222202004013572,0,文化,"[3, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34..."
2,6679940295825555980,0,文化,"[37, 38, 39, 40, 7, 41, 26, 28, 28, 14]"
3,6676963806968545799,0,文化,"[42, 31, 43, 44, 45, 46, 47, 48, 49, 50, 51, 5..."
4,6664825683656573448,0,文化,"[29, 28, 28, 28, 30, 60, 61, 62, 47, 63, 30, 1..."
...,...,...,...,...
56816,6691485644243337735,8,国际,"[537, 2192, 753, 287, 303, 1116, 490, 1776, 70..."
56817,6691479002646839822,8,国际,"[518, 729, 735, 132, 157, 183, 39, 1122, 81, 1..."
56818,6691483762988941835,8,国际,"[55, 728, 807, 1297, 818, 906, 55, 183, 419, 9..."
56819,6691481494600286727,8,国际,"[157, 183, 61, 1490, 131, 112, 1292, 259, 2252..."


In [24]:
import pickle
f = open('char_df.pkl','wb')
pickle.dump(df,f)
f.close()
f = open('char2idx.pkl','wb')
pickle.dump(char2idx,f)
f.close()
f = open('idx2char.pkl','wb')
pickle.dump(idx2char,f)
f.close()